In [1]:
!pip install numpy
!pip install opencv-python
!pip install torch==2.3.1
!pip install tqdm
!pip install albumentations
!pip install transformers
!pip install torchvision
!pip install torch_optimizer
!pip install segmentation_models_pytorch
!pip install glob
!pip install pathlib
!pip install ssl
!pip install pandas
!pip install scikit-learn
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 14, in <module>
        File "/home/anton/anaconda3/lib/python3.11/site-packages/setuptools/__init__.py", line 27, in <module>
          from .dist import Distribution
        File "/home/anton/anaconda3/lib/python3.11/site-packages/setuptools/dist.py", line 18, in <module>
          from . import (
        File "/home/anton/anac

In [69]:
import numpy as np
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys, os
import math
import random
import albumentations as A
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
from collections import OrderedDict
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch_optimizer import Ranger
print(Variable(torch.tensor(1.0)))

tensor(1.)


In [70]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random_state = 69
img_shape = (512, 512)
epochs = 10
early_stopping = 50
train_batch_size = 4
test_batch_size = 8
images_dir = 'new_images'
masks_dir = 'new_masks'
AUG_PROB = 0.75
AUG = True
GRAD_ACC = 1
use_multitask = False

In [71]:
def set_random_seed(seed: int = 2222, deterministic: bool = False):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = deterministic
    
set_random_seed(random_state)

In [72]:
def read_image(filepath):
    image = Image.open(filepath)
    image = np.array(image)
    image = cv2.resize(image, (512, 512))
    return image


def read_mask(filepath):
    image = Image.open(filepath).convert('L')
    image = np.array(image).astype(np.float64)
    image /= image.max()
    image = image.astype(np.uint8)
    image = cv2.resize(image, (512, 512))
    return image

def mape(counts, counts_true):
    return 100 / len(counts) * sum([abs(counts_true[i] - counts[i]) / counts_true[i]
                                        for i in range(len(counts))])

In [73]:
transforms_train = A.Compose([
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=AUG_PROB),

#     A.OneOf([
#         A.OpticalDistortion(distort_limit=1.0),
#         A.GridDistortion(num_steps=5, distort_limit=1.),
#         A.ElasticTransform(alpha=3),
#     ], p=AUG_PROB),

#     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
    A.Resize(img_shape[0], img_shape[1]),
#     A.CoarseDropout(max_holes=16, max_height=16, max_width=16, min_holes=1, min_height=2, min_width=2, p=AUG_PROB),    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transforms_val = A.Compose([
    A.Resize(img_shape[0], img_shape[1]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_for_masks = A.Compose([
    A.Resize(img_shape[0], img_shape[1])
])

if not AUG:
    transforms_train = transforms_val

/home/anton/anaconda3/lib/python3.11/site-packages/albumentations/augmentations/blur/transforms.py:189: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [74]:
from scipy import ndimage

def count_pipes_on_single_mask(mask: np.ndarray, minimum_filter_size: int = 3, std_coef: float = 2.5) -> int:
    filtered_mask = ndimage.minimum_filter(mask, size=minimum_filter_size)
    filtered_mask = np.round(filtered_mask).astype(bool)
    size = filtered_mask.shape
    visited = [[False] * size[0] for _ in range(size[1])]
    components_list = []
    for y in range(size[1]):
        for x in range(size[0]):
            if mask[y][x] < 0.5:
                continue
            if visited[y][x]:
                continue
        queue = [(x, y)]
        visited[y][x] = True
        components_list.append(0)
        while queue:
            components_list[-1] += 1
            current_x, current_y = queue.pop(0)
            for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)):
                next_x, next_y = current_x + dx, current_y + dy
                if not (0 <= next_y < size[0] and 0 <= next_x < size[1]):
                    continue
                if mask[next_y, next_x] and not visited[next_y][next_x]:
                    queue.append((next_x, next_y))
 
    components_array = np.array(components_list)
    mean, std = components_array.mean(), components_array.std()
 
    return (components_array > mean + std_coef * std).sum()

from utils import testing_model
from utils import bfs


In [75]:
import segmentation_models_pytorch as smp

autocast = torch.cuda.amp.autocast(enabled=True, dtype=torch.half)

In [76]:
from glob import glob
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [77]:
def inference(models, images_path, output_path='output_masks', save_npz=False):
    #inference of the model
    counts = []
    masks = {}
    if not Path(output_path).exists():
        os.mkdir(output_path)
    for i, image_path in tqdm(enumerate(images_path), total=len(images_path), position=0):
        image = read_image(image_path)
        super_output = np.zeros((img_shape[0], img_shape[1]))
        count = -1
        for model in models:
            model.eval()
            with torch.no_grad():
                if use_multitask:
                    output, proba, types = testing_model(model, image)
                else:
                    output, proba = testing_model(model, image)
                    
                super_output += (proba / len(models))
                
#         cnts = count_pipes_on_single_mask(super_output)   
        super_output = super_output.round()
        comps = bfs(super_output)
        
        counts.append(len(comps))
        masks[image_path.split('/')[-1][:-4]] = super_output * 255

        if output_path[-1] == '0':
            super_output = super_output.reshape(1, super_output.shape[0], super_output.shape[1])
            super_output = np.concatenate([super_output, super_output, super_output])
            super_output = np.transpose(super_output, (1, 2, 0))
            super_output *= 200
            super_output[:, :, 1] -= 20
            
        elif output_path[-1] == '1':
            super_output = super_output.reshape(1, super_output.shape[0], super_output.shape[1])
            super_output = np.concatenate([super_output, super_output, super_output])
            super_output = np.transpose(super_output, (1, 2, 0))
            super_output *= 130
            super_output[:, :, 1] -= 50
        else:
            super_output *= 255
        cv2.imwrite(os.path.join(output_path, image_path.split('/')[-1][:-4] + '.png'), 
                    super_output)

    if save_npz:
        np.savez('masks.npz', **masks)
    return counts


In [78]:
class MultiTaskModel(nn.Module):
    def __init__(self, unet):
        super(MultiTaskModel, self).__init__()
        self.unet = unet
        
        classification_blocks = [
            nn.Conv2d(256, 512, 3, 3),
            nn.BatchNorm2d(512),
            nn.SiLU(),
            nn.Conv2d(512, 128, 3, 3),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.Conv2d(128, 64, 2, 2),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AdaptiveAvgPool2d((4, 4)),
            nn.Flatten(),
            nn.Linear(1024, 2)
        ]
        
        regression_blocks = [
            nn.Conv2d(256, 128, 3, 3),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.Conv2d(128, 64, 2, 2),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AdaptiveAvgPool2d((3, 3)),
            nn.Flatten(),
            nn.Linear(576, 1)
        ]
        
        self.classification_head = nn.Sequential(*classification_blocks)
        self.regression_head = nn.Sequential(*regression_blocks)
        
    def forward(self, x):
        embedding = self.unet.encoder.conv1(x)
        embedding = self.unet.encoder.bn1(embedding)
        embedding = self.unet.encoder.relu(embedding)
        embedding = self.unet.encoder.maxpool(embedding)
        embedding = self.unet.encoder.layer1(embedding)
        embedding = self.unet.encoder.layer2(embedding)
        embedding = self.unet.encoder.layer3(embedding)
        
        num_of_pipes = self.regression_head(embedding)
        type_of_pipes = self.classification_head(embedding)
        
        mask = self.unet(x)
        return mask, num_of_pipes, type_of_pipes

In [79]:
from glob import glob
from pathlib import Path

do_pseudolabeling = False

model_pathes = glob('trained_models/fold 1/*.pt') + glob('trained_models/fold 2/*.pt') + glob('train_models/fold 3/*.pt') + glob('trained_models/fold 4/*.pt') + glob('trained_models/fold 5/*.pt')
models = []
model_pathes.sort()
print(model_pathes)

for model_path in model_pathes[:1]:
    
    model = torch.load(model_path if not do_pseudolabeling else 'psl/model_resnet18_lovasz_05_05_70.pt', map_location=device)
    model.eval()
    models.append(model)

print(len(models))
# model = torch.load('model_resnet18_lovasz_05_05_110.pt' if not do_pseudolabeling else 'psl/model_resnet18_lovasz_05_05_110.pt', map_location=device)
t_images_path0 = glob('severstal-pipes/test/0/*.JPG')
t_images_path0.sort()
t_images_path1 = glob('severstal-pipes/test/1/*.JPG')
t_images_path1.sort()
print(t_images_path0)
print(t_images_path1)
if not Path('output_masks').exists():
    os.mkdir('output_masks')
counts0 = inference(models, t_images_path0, output_path='output_masks/0')
counts1 = inference(models, t_images_path1, output_path='output_masks/1')
counts0, counts1

['trained_models/fold 1/model_resnet18_lovasz_05_05_120.pt', 'trained_models/fold 1/model_resnet18_lovasz_05_05_140.pt', 'trained_models/fold 1/model_resnet18_lovasz_05_05_300.pt', 'trained_models/fold 2/model_resnet18_lovasz_05_05_110.pt', 'trained_models/fold 4/model_resnet18_lovasz_05_05_110.pt', 'trained_models/fold 5/model_resnet18_lovasz_05_05_110.pt']
1
['severstal-pipes/test/0/IMG_2947.JPG', 'severstal-pipes/test/0/IMG_2955.JPG', 'severstal-pipes/test/0/IMG_2956.JPG']
['severstal-pipes/test/1/IMG_2930.JPG', 'severstal-pipes/test/1/IMG_2931.JPG', 'severstal-pipes/test/1/IMG_2968.JPG']


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

([131, 104, 106], [278, 294, 186])

In [80]:
import pandas as pd
# counts0 = [127, 104, 104]
# counts1 = [264, 264, 185]
submission0 = pd.DataFrame()
submission0['img_name'] = ['/'.join(name.split('/')[:])[:-4]+'.jpg' for name in t_images_path0]
submission0['annot_path'] = ['output_masks/' + (name.split('/')[-1])[:-4]+'.png' for name in t_images_path0]
submission0['pipe_class'] = ['0'] * len(t_images_path0)
submission0['pipe_count'] = counts0

submission1 = pd.DataFrame()
submission1['img_name'] = ['/'.join(name.split('/')[:])[:-4]+'.jpg' for name in t_images_path1]
submission1['annot_path'] = ['output_masks/' + (name.split('/')[-1])[:-4]+'.png' for name in t_images_path1]
submission1['pipe_class'] = ['1'] * len(t_images_path1)
submission1['pipe_count'] = counts1

# submission1.iloc[1, 3] = 264

submission = pd.concat((submission0, submission1))
submission['temp'] = [submission['img_name'].iloc[row].split('/')[-1]
                    for row in range(len(submission))]
submission = submission.sort_values('temp')
submission = submission.drop(columns=['temp'])
submission

,img_name,annot_path,pipe_class,pipe_count
0,severstal-pipes/test/1/IMG_2930.jpg,output_masks/IMG_2930.png,1,278
1,severstal-pipes/test/1/IMG_2931.jpg,output_masks/IMG_2931.png,1,294
0,severstal-pipes/test/0/IMG_2947.jpg,output_masks/IMG_2947.png,0,131
1,severstal-pipes/test/0/IMG_2955.jpg,output_masks/IMG_2955.png,0,104
2,severstal-pipes/test/0/IMG_2956.jpg,output_masks/IMG_2956.png,0,106
2,severstal-pipes/test/1/IMG_2968.jpg,output_masks/IMG_2968.png,1,186


In [81]:
submission.to_csv('submission.csv', index=False, sep=";")

In [82]:
100 - mape(counts0 + counts1, [127, 104, 104, 264, 264, 185])

96.28668492841722

In [83]:
100 - mape([127, 109, 106, 268, 264, 181], [127, 104, 104, 264, 264, 185])

98.26531951531952

In [84]:
100 - mape([131, 103, 105, 255, 288, 169], [127, 104, 104, 264, 264, 185])

95.6297780215103